# Sprint Project: Data Preprocessing
Prepared by `Kuhgi Jotojot`

## Prerequisites

In [302]:
# pip install scikit-learn

In [303]:
# pip install pandas

In [304]:
# general libraries
import time
import warnings
import numpy as np
import pandas as pd

# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [305]:
# OneHotEncoder for categorical data
from sklearn.preprocessing import OneHotEncoder

In [306]:
df = pd.read_csv('data/Family Income and Expenditure.csv')
df.head()

,Total Household Income,Region,Total Food Expenditure,Main Source of Income,Agricultural Household indicator,Bread and Cereals Expenditure,Total Rice Expenditure,Meat Expenditure,Total Fish and marine products Expenditure,Fruit Expenditure,...,Number of Refrigerator/Freezer,Number of Washing Machine,Number of Airconditioner,"Number of Car, Jeep, Van",Number of Landline/wireless telephones,Number of Cellular phone,Number of Personal Computer,Number of Stove with Oven/Gas Range,Number of Motorized Banca,Number of Motorcycle/Tricycle
0,11285,VII - Central Visayas,14709,Other sources of Income,1,4690,91,248,1759,236,...,0,0,0,0,0,0,0,0,0,0
1,11988,CAR,6781,Other sources of Income,0,4175,4175,930,360,581,...,0,0,0,0,0,0,0,0,0,0
2,12039,V - Bicol Region,9465,Other sources of Income,0,3176,3041,150,1410,336,...,0,0,0,0,0,0,0,0,0,0
3,12141,II - Cagayan Valley,6500,Other sources of Income,0,1851,1140,1310,1318,330,...,0,0,0,0,0,0,0,0,0,0
4,12911,VII - Central Visayas,15909,Enterpreneurial Activities,1,6716,96,1030,3316,285,...,0,0,0,0,0,0,0,0,0,0


## Data Cleaning

### 1. Dropping Duplicates

In [307]:
df = df.drop_duplicates()

### 2. Structural Fix (Renaming Column Names)

In [308]:
for col in df.columns:
  new_col = col.replace(' ', '_')
  df.rename(columns={col:new_col}, inplace=True)

### 3. Structural Fix (Substituting Null Values)

In [309]:
value = "No Occupation"
df['Household_Head_Occupation'] = df['Household_Head_Occupation'].fillna(value)

In [310]:
value = "Not Employed"
df['Household_Head_Class_of_Worker'] = df['Household_Head_Class_of_Worker'].fillna(value)

In [311]:
value = "No Toilet Facilities"
df['Toilet_Facilities'] = df['Toilet_Facilities'].fillna(value)

### 4. Fixing Misspelling in Categories

In [312]:
df['Main_Source_of_Income'] = df['Main_Source_of_Income'].map({'Enterpreneurial Activities': 'Entrepreneurial Activities'})
df['Household_Head_Class_of_Worker'] = df['Household_Head_Class_of_Worker'].map({'Self-employed wihout any employee': 'Self-employed without any employee'})
df['Type_of_Walls'] = df['Type_of_Walls'].map({'NOt applicable': 'Not Applicable'})

## Categorical Fields Processing

In [313]:
# Getting all object fields in df (categorical feature)
object_fields = df.select_dtypes(include=['object']).columns.tolist()

In [314]:
# Create a dictionary to store field names and their category counts
category_counts = {field: df[field].nunique() for field in object_fields}

# Convert the dictionary into a DataFrame
category_counts_df = pd.DataFrame(list(category_counts.items()), columns=['Field', 'Number_of_Categories'])

# Display the resulting DataFrame
category_counts_df[category_counts_df['Number_of_Categories'] >= 10]

,Field,Number_of_Categories
0,Region,17
4,Household_Head_Highest_Grade_Completed,46
6,Household_Head_Occupation,379
14,Main_Source_of_Water_Supply,11


### Plan for Categorical Values

* Region - Mean Encoding (Average the household income per region. Make that as the region feature instead [Average_Income_per_Capita])
* Household Head Highest Grade Completed - feature hashing  and then OneHotEncode
* Household_Head_Occupation - Feature Hashing and then Mean Encoding (Average the occupational income per occupation. Make that as the occupation feature instead [Average_Income_per_Occupation_Group])
* Main_Source_of_Water_Supply - Generalization to 6 categories and then OneHotEncode

### 1. Region -> Average Income per Capita

In [315]:
total_income_per_region = df.groupby('Region')['Total_Household_Income'].mean().reset_index()
total_income_per_region.columns = ['Region', 'Average_Income_per_Capita']
total_income_per_region['Average_Income_per_Capita'] = total_income_per_region['Average_Income_per_Capita'].round(2)
region_income_dict = total_income_per_region.set_index('Region')['Average_Income_per_Capita'].to_dict()
df['Average_Income_per_Capita'] = df['Region'].map(region_income_dict)

### 2. Main_Source_of_Water_Supply Generalization

In [316]:
# Generalize the categories for 'Main_Source_of_Water_Supply'
water_supply_mapping = {
    'Dug well': 'Well',
    'Protected spring, river, stream, etc': 'Natural Source',
    'Shared, tubed/piped deep well': 'Shared Source',
    'Shared, faucet, community water system': 'Shared Source',
    'Unprotected spring, river, stream, etc': 'Natural Source',
    'Own use, faucet, community water system': 'Private Source',
    'Own use, tubed/piped deep well': 'Private Source',
    'Tubed/piped shallow well': 'Well',
    'Lake, river, rain and others': 'Natural Source',
    'Peddler': 'Peddler',
    'Others': 'Other'
}

# Apply the mapping to the column
df['Main_Source_of_Water_Supply'] = df['Main_Source_of_Water_Supply'].map(water_supply_mapping)

### 3. Household_Head_Occupation -> Average Incomer per Occupation Group

In [317]:
# Final integrated occupation mapping function with all provided classifications
def map_occupation_category_final(occupation):
    if pd.isnull(occupation):
        return "Unspecified"

    occupation = occupation.lower()

    if any(keyword in occupation for keyword in [
        "teacher", "professor", "educat", "instructor", "school principal",
        "science and mathematics teaching", "other teaching professionals",
        "other social science professionals", "accountant", "auditor", "architect",
        "appraiser", "valuer", "other business professionals"
    ]):
        return "Professionals"

    elif any(keyword in occupation for keyword in [
        "engineer", "chemist", "scientist", "r&d", "development", "technologist",
        "research", "geodetic", "economist", "statistician", "draftsman", "planner",
        "programmer", "computer professional"
    ]):
        return "Science & Engineering"

    elif any(keyword in occupation for keyword in [
        "technician", "tech", "associate", "assistant", "midwife", "nutritionist",
        "draftsmen", "computer assistant", "library", "librarian", "lab",
        "building and fire inspector"
    ]):
        return "Technicians and Associate Professionals"

    elif any(keyword in occupation for keyword in [
        "farmer", "fisher", "fish-farm", "seaweed", "livestock", "agricultur", "forester",
        "horticulturist", "planter", "duck raiser", "ornamental plant grower", "trapper",
        "animal raiser", "minor forest", "plant grower", "aqua products cultivator"
    ]):
        return "Agricultural, Forestry & Fishery Workers"

    elif any(keyword in occupation for keyword in [
        "carpenter", "plumber", "electrician", "welder", "mechanic", "mason", "blacksmith",
        "construction", "craft", "fitter", "assembler", "tailor", "sewer", "textile", "shoemaker",
        "upholster", "cabinet", "repairer", "molder", "coremaker", "grinder", "polisher", "coater",
        "roofer", "floor layer", "tile setter", "glass", "woodworker", "bookbinder", "metal drawer",
        "metal worker", "jewelry", "stone splitter", "potter", "tanner", "leather", "weaver", "bamboo",
        "rattan", "insulation", "pressman", "block", "screen", "printer", "food preserver", "dairy products maker",
        "tobacco preparer", "builder", "charcoal maker", "tool-maker", "musical instrument maker", "wood treater",
        "compositor", "typesetter", "furniture maker", "varnisher", "painter", "structural-metal preparer",
        "food and beverage taster", "building frame and related trades", "miners and quarry workers", "fiber preparers"
    ]):
        return "Craft and Related Trades Workers"

    elif any(keyword in occupation for keyword in [
        "driver", "operator", "assembler", "machine", "truck", "crane", "loader", "hoist",
        "tractor", "vehicle", "pilot", "deck officer", "plant operator", "production machine",
        "ship", "transport conductor", "lineman", "cable splicer", "well driller", "rigger", "blaster"
    ]):
        return "Plant & Machine Operators"

    elif any(keyword in occupation for keyword in [
        "laborer", "helper", "vendor", "cleaner", "sweeper", "collector", "launderer", "packer",
        "janitor", "caretaker", "watchperson", "door-to-door", "messenger", "porter", "housekeeper",
        "freight handler", "shoe cleaning", "elementary occupation"
    ]):
        return "Elementary Occupations"

    elif any(keyword in occupation for keyword in [
        "manager", "executive", "chief", "director", "administrator", "supervisor", "proprietor",
        "head", "staff officer", "government official", "officer", "licensing", "commanding",
        "justices", "legislative official", "social benefits official", "senior official",
        "government tax and excise"
    ]):
        return "Managers"

    elif any(keyword in occupation for keyword in [
        "clerk", "secretary", "bookkeeper", "typist", "cashier", "teller", "encoder",
        "receptionist", "mail carrier", "information clerk", "word processor", "secretaries"
    ]):
        return "Clerical Support Workers"

    elif any(keyword in occupation for keyword in [
        "sales", "waiter", "bartender", "beautician", "hairdresser", "concessionaire",
        "service worker", "guide", "valet", "cook", "butcher", "fishmonger", "baker", "vendor",
        "housekeeper", "personal care", "caregiver", "attendant", "bookmaker", "croupier",
        "estate agent", "insurance representative", "buyer", "travel consultant", "trade broker",
        "child care", "undertaker", "embalmer", "pawnbroker", "personal services", "service and related workers"
    ]):
        return "Service and Sales Workers"

    elif any(keyword in occupation for keyword in [
        "artist", "musician", "composer", "dancer", "designer", "photographer", "sculptor",
        "model", "performer", "writer", "journalist", "radio", "announcer", "athletes and related workers", "photographic and related workers"
    ]):
        return "Creative, Arts & Media"

    elif any(keyword in occupation for keyword in [
        "police", "soldier", "military", "guard", "firefighter", "uniformed", "combat",
        "prison", "customs", "immigration", "detective", "protective services", "enlisted personnel"
    ]):
        return "Armed Forces & Uniformed Personnel"

    elif any(keyword in occupation for keyword in [
        "lawyer", "doctor", "nurse", "dentist", "pharmacist", "therapist", "optician",
        "midwife", "medical", "health professional", "dietitian", "veterinarian",
        "faith healer", "traditional medicine", "religious", "social worker", "professional midwives", "social work professionals"
    ]):
        return "Health & Legal Professionals"

    else:
        return "Other / Unclassified"


In [318]:
df["Household_Head_Occupation"] = df["Household_Head_Occupation"].apply(map_occupation_category_final)

In [319]:
total_income_per_occupation_group = df.groupby('Household_Head_Occupation')['Total_Household_Income'].mean().reset_index()
total_income_per_occupation_group.columns = ['Occupation Group', 'Average_Income_per_Occupation_Group']
total_income_per_occupation_group['Average_Income_per_Occupation_Group'] = total_income_per_occupation_group['Average_Income_per_Occupation_Group'].round(2)
occupation_group_income_dict = total_income_per_occupation_group.set_index('Occupation Group')['Average_Income_per_Occupation_Group'].to_dict()
occupation_group_income_dict

df['Average_Income_per_Occupation_Group'] = df['Household_Head_Occupation'].map(occupation_group_income_dict)

### 4. Household_Head_Highest_Grade_Completed Features Hashing

In [320]:
education_mapping = {
    # No education
    "No Grade Completed": "No Formal Education",

    # Preschool & Elementary
    "Preschool": "Preschool/Elementary",
    "Grade 1": "Preschool/Elementary",
    "Grade 2": "Preschool/Elementary",
    "Grade 3": "Preschool/Elementary",
    "Grade 4": "Preschool/Elementary",
    "Grade 5": "Preschool/Elementary",
    "Grade 6": "Preschool/Elementary",
    "Elementary Graduate": "Preschool/Elementary",

    # High School (Incomplete and Graduate)
    "First Year High School": "High School",
    "Second Year High School": "High School",
    "Third Year High School": "High School",
    "High School Graduate": "High School",

    # I will assume these two as high school graduate in order to reduce the categories
    "Other Programs of Education at the Third Level, First Stage, of the Type that Leads to a Baccalaureate or First University/Professional Degree (Higher Education Level, First Stage, or Collegiate Education Level)": "High School",
    "Other Programs in Education at the Third Level, First Stage, of the Type that Leads to an Award not Equivalent to a First University or Baccalaureate Degree": "High School",

    # College years
    "First Year College": "College",
    "Second Year College": "College",
    "Third Year College": "College",
    "Fourth Year College": "College",

    # Post-secondary non-degree
    "First Year Post Secondary": "Post-Secondary (Non-Degree)",
    "Second Year Post Secondary": "Post-Secondary (Non-Degree)",

    # College graduate or higher
    "Post Baccalaureate": "College",
    "Other Programs of Education at the Third Level, First Stage, of the Type that Leads to a Baccalaureate or First University/Professional Degree (Higher Education Level, First Stage, or Collegiate Education Level)": "College Graduate",
    "Other Programs in Education at the Third Level, First Stage, of the Type that Leads to an Award not Equivalent to a First University or Baccalaureate Degree": "College Graduate",

    # General tertiary fields
    "Teacher Training and Education Sciences Programs": "Tertiary (General)",
    "Business and Administration Programs": "Tertiary (General)",
    "Humanities Programs": "Tertiary (General)",
    "Social and Behavioral Science Programs": "Tertiary (General)",
    "Mathematics and Statistics Programs": "Tertiary (General)",
    "Arts Programs": "Tertiary (General)",
    "Law Programs": "Tertiary (General)",
    "Social Services Programs": "Tertiary (General)",
    "Journalism and Information Programs": "Tertiary (General)",
    "Basic Programs": "Tertiary (General)",

    # STEM/Technical fields
    "Engineering and Engineering Trades Programs": "Tertiary (STEM/Tech)",
    "Engineering and Engineering trades Programs": "Tertiary (STEM/Tech)",
    "Health Programs": "Tertiary (STEM/Tech)",
    "Veterinary Programs": "Tertiary (STEM/Tech)",
    "Agriculture, Forestry, and Fishery Programs": "Tertiary (STEM/Tech)",
    "Life Sciences Programs": "Tertiary (STEM/Tech)",
    "Physical Sciences Programs": "Tertiary (STEM/Tech)",
    "Architecture and Building Programs": "Tertiary (STEM/Tech)",
    "Manufacturing and Processing Programs": "Tertiary (STEM/Tech)",
    "Environmental Protection Programs": "Tertiary (STEM/Tech)",

    # Vocational/Technical Services
    "Security Services Programs": "Vocational/Technical",
    "Personal Services Programs": "Vocational/Technical",
    "Transport Services Programs": "Vocational/Technical",
}

df["Household_Head_Highest_Grade_Completed"] = df["Household_Head_Highest_Grade_Completed"].map(education_mapping)

In [321]:
# Drop 'Region' and 'Household_Head_Occupation'
df = df.drop(['Region', 'Household_Head_Occupation'], axis=1)

## Converting Binary Categorical Variables into Binary Numerical Variables

In [322]:
df['Household_Head_Sex'] = df['Household_Head_Sex'].map({'Female': 0, 'Male' : 1})

In [323]:
df['Household_Head_Job_or_Business_Indicator'] = df['Household_Head_Job_or_Business_Indicator'].map({'No Job/Business': 0, 'With Job/Business' : 1})

## OneHotEncoding Categorical Variables

### Pseudocode for OneHotEncoding

* Initiate a new df (pre_encoded_df) 
* Get all the categorical features in the main df and append them to pre_encoded_df
* df_encoded = pd.get_dummies(df, dtype=int)
* concat df_encoded with main df
* For every categorical features in object fields...
  * Drop it from the main df

In [324]:
pre_encoded_df = pd.DataFrame()

# Get a list of all categorical features in df
object_fields = df.select_dtypes(include=['object']).columns.tolist()

# Append all the features in object_fields into 'pre_encooded_df'
for column in object_fields:
    pre_encoded_df[column] = df[column]

# pre_encoded_df

df_encoded = pd.get_dummies(pre_encoded_df, dtype=int)

for col in df_encoded.columns:
  new_col = col.replace(' ', '_')
  df_encoded.rename(columns={col:new_col}, inplace=True)

df = pd.concat([df, df_encoded], axis=1)

for column in object_fields:
  df = df.drop([column], axis=1)

## Saving Final Dataset into New CSV File

In [325]:
df.to_csv('data/Family_Income_and_Expenditure_Revised.csv', index=False)

## Trash Code Lines Below (consider deleting)

In [36]:
df.head()

,Total_Household_Income,Total_Food_Expenditure,Main_Source_of_Income,Agricultural_Household_indicator,Bread_and_Cereals_Expenditure,Total_Rice_Expenditure,Meat_Expenditure,Total_Fish_and__marine_products_Expenditure,Fruit_Expenditure,Vegetables_Expenditure,...,"Number_of_Car,_Jeep,_Van",Number_of_Landline/wireless_telephones,Number_of_Cellular_phone,Number_of_Personal_Computer,Number_of_Stove_with_Oven/Gas_Range,Number_of_Motorized_Banca,Number_of_Motorcycle/Tricycle,Average_Income_per_Capita,Main_Source_of_Water_Supply_Generalized,Average_Income_per_Occupation_Group
0,11285,14709,Other sources of Income,1,4690,91,248,1759,236,1335,...,0,0,0,0,0,0,0,234909.31,Well,152836.44
1,11988,6781,Other sources of Income,0,4175,4175,930,360,581,525,...,0,0,0,0,0,0,0,269540.48,Natural Source,285022.93
2,12039,9465,Other sources of Income,0,3176,3041,150,1410,336,241,...,0,0,0,0,0,0,0,186105.49,Shared Source,152836.44
3,12141,6500,Other sources of Income,0,1851,1140,1310,1318,330,560,...,0,0,0,0,0,0,0,236778.22,Shared Source,285022.93
4,12911,15909,Enterpreneurial Activities,1,6716,96,1030,3316,285,1831,...,0,0,0,0,0,0,0,234909.31,Shared Source,168933.22


In [37]:
# Getting all object fields in df (categorical feature)
object_fields = df.select_dtypes(include=['object']).columns.tolist()
object_fields

['Main_Source_of_Income',
 'Household_Head_Sex',
 'Household_Head_Marital_Status',
 'Household_Head_Highest_Grade_Completed',
 'Household_Head_Job_or_Business_Indicator',
 'Household_Head_Class_of_Worker',
 'Type_of_Household',
 'Type_of_Building/House',
 'Type_of_Roof',
 'Type_of_Walls',
 'Tenure_Status',
 'Toilet_Facilities',
 'Main_Source_of_Water_Supply',
 'Main_Source_of_Water_Supply_Generalized']

In [ ]:
# Define categorical variables
categorical_variables_list = ['Region', 'Main Source of Income', 'Agricultural Household indicator', 'Household Head Sex', 'Household Head Marital Status', 'Household Head Highest Grade Completed', 'Household Head Occupation', 'Household Head Class of Worker', 'Type of Household', 'Type of Building/House', 'Type of Roof', 'Type of Walls', 'Tenure Status', 'Toilet Facilities', 'Main Source of Water Supply']
# OneHotEncoding some of these categorical variables might result into 'curse of dimensionality'
# Some categorical values in this dataset have a lot of categories

# Should we use PCA in these categorical values?
# Another issue we will bring up: Incomplete records
# By dropping the incomplete records with dropna() function, are we misrepresenting the data?
# We only dropped like 8956 records with the method

In [ ]:
# # Getting all the new object fields in df (categorical feature)
# object_fields = df.select_dtypes(include=['object']).columns.tolist()

# # Create a dictionary to store field names and their category counts
# category_counts = {field: df[field].nunique() for field in object_fields}

# # Convert the dictionary into a DataFrame
# category_counts_df = pd.DataFrame(list(category_counts.items()), columns=['Field', 'Number_of_Categories'])

# # Display the resulting DataFrame
# category_counts_df

Links:
https://www.datacamp.com/tutorial/one-hot-encoding-python-tutorial
